# Generating a final answer from a missing descriptions

In [ ]:
! pip install groq -q

In [ ]:
from google.colab import userdata
import os
from groq import Groq
import pandas as pd
import re
import ast
import json
from google.colab import userdata

api_key = userdata.get('GROQ_API_KEY')
model= "llama3-70b-8192"
client = Groq(api_key=api_key)

In [ ]:
df = pd.read_csv("/content/adas_systems_step-01.csv")
df

,System,Description Modified,Missing Rows
0,Adaptive Cruise Control,# Technical Details\n**Radar and Camera Sensor...,"[{'section': '# Technical Details', 'missing_l..."
1,Adaptive Cruise Control,# Technical Details\n\n\n**Electronic Control ...,"[{'section': '# Technical Details', 'missing_l..."
2,Adaptive Cruise Control,# Technical Details\n\n\n**Radar and Camera Se...,"[{'section': '# Technical Details', 'missing_l..."
3,Adaptive Cruise Control,# Technical Details\n\n\n**Radar and Camera Se...,"[{'section': '# Technical Details', 'missing_l..."
4,Adaptive Cruise Control,# Technical Details\n\n**Radar and Camera Sens...,"[{'section': '# Technical Details', 'missing_l..."
...,...,...,...
2995,Blind Spot Monitoring,# Technical Details\n\n\n**Sensor Suite**: The...,"[{'section': '# Technical Details', 'missing_l..."
2996,Blind Spot Monitoring,# Technical Details\n\n**Sensor Suite**: The B...,"[{'section': '# Technical Details', 'missing_l..."
2997,Blind Spot Monitoring,# Technical Details\n\n\n\t* Radar sensors (24...,"[{'section': '# Technical Details', 'missing_l..."
2998,Blind Spot Monitoring,# Technical Details\n\n**Sensor Suite**: The B...,"[{'section': '# Technical Details', 'missing_l..."


In [ ]:
df = df.iloc[551:651]
df

,System,Description Modified,Missing Rows
801,Adaptive Cruise Control,# Technical Details\n\n\n**Radar and Camera Se...,"[{'section': '# Technical Details', 'missing_l..."
802,Adaptive Cruise Control,# Technical Details\n\n**Radar and Camera Sens...,"[{'section': '# Technical Details', 'missing_l..."
803,Adaptive Cruise Control,# Technical Details\n\n**Radar and Camera Sens...,"[{'section': '# Technical Details', 'missing_l..."
804,Adaptive Cruise Control,# Technical Details\n\n\n**Electronic Control ...,"[{'section': '# Technical Details', 'missing_l..."
805,Adaptive Cruise Control,# Technical Details\n\n**Electronic Control Un...,"[{'section': '# Technical Details', 'missing_l..."
...,...,...,...
896,Adaptive Cruise Control,# Technical Details\n\n**Radar and Camera Sens...,"[{'section': '# Technical Details', 'missing_l..."
897,Adaptive Cruise Control,# Technical Details\n\n**Powertrain and Brakin...,"[{'section': '# Technical Details', 'missing_l..."
898,Adaptive Cruise Control,# Technical Details\n\n\n**Radar and Camera Se...,"[{'section': '# Technical Details', 'missing_l..."
899,Adaptive Cruise Control,# Technical Details\n\n\n**Electronic Control ...,"[{'section': '# Technical Details', 'missing_l..."


In [ ]:
def parse_missing_rows(row):
    try:
        # Parse the string into a list of dictionaries
        missing_list = ast.literal_eval(row['Missing Rows'])

        # Extract non-empty missing_lines and their corresponding sections
        missing_details = []
        for item in missing_list:
            if item['missing_line']:
                missing_details.append(f"{item['section']}: {item['missing_line']}")

        # Combine into a paragraph
        if missing_details:
            paragraph = " ".join(missing_details)
        else:
            paragraph = "No missing lines in this section."

        return paragraph
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing row {row['id']}: {e}")
        return "Error parsing missing rows."


In [ ]:
df['Parsed Missing Rows'] = df.apply(parse_missing_rows, axis=1)

# Display the updated DataFrame
print("Updated DataFrame:")
df

Updated DataFrame:


<ipython-input-40-60ff3deb3cc3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Parsed Missing Rows'] = df.apply(parse_missing_rows, axis=1)


,System,Description Modified,Missing Rows,Parsed Missing Rows
801,Adaptive Cruise Control,# Technical Details\n\n\n**Radar and Camera Se...,"[{'section': '# Technical Details', 'missing_l...",# Technical Details: 2. **Electronic Control U...
802,Adaptive Cruise Control,# Technical Details\n\n**Radar and Camera Sens...,"[{'section': '# Technical Details', 'missing_l...",# Technical Details: 2. **Electronic Control U...
803,Adaptive Cruise Control,# Technical Details\n\n**Radar and Camera Sens...,"[{'section': '# Technical Details', 'missing_l...",# Technical Details: 2. **Electronic Control U...
804,Adaptive Cruise Control,# Technical Details\n\n\n**Electronic Control ...,"[{'section': '# Technical Details', 'missing_l...",# Technical Details: 1. **Radar and Camera Sen...
805,Adaptive Cruise Control,# Technical Details\n\n**Electronic Control Un...,"[{'section': '# Technical Details', 'missing_l...",# Technical Details: 4. **GPS and INERTIAL Mea...
...,...,...,...,...
896,Adaptive Cruise Control,# Technical Details\n\n**Radar and Camera Sens...,"[{'section': '# Technical Details', 'missing_l...",# Operational Steps: 4. **Distance and Speed C...
897,Adaptive Cruise Control,# Technical Details\n\n**Powertrain and Brakin...,"[{'section': '# Technical Details', 'missing_l...",# Technical Details: 1. **Radar and Camera Sen...
898,Adaptive Cruise Control,# Technical Details\n\n\n**Radar and Camera Se...,"[{'section': '# Technical Details', 'missing_l...",# Technical Details: 2. **Electronic Control U...
899,Adaptive Cruise Control,# Technical Details\n\n\n**Electronic Control ...,"[{'section': '# Technical Details', 'missing_l...",# Technical Details: 1. **Radar and Camera Sen...


In [ ]:
print("It looks like you missed some important details in your initial response:\n", "\n".join(df['Parsed Missing Rows'][900].split("#")), "\n Let me provide you with a complete description of the ",df['System'][900]," system.")

It looks like you missed some important details in your initial response:
 
 Technical Details: 1. **Radar and Camera Sensors**: The ACC system employs a combination of radar and camera sensors to detect and track the distance, speed, and trajectory of the preceding vehicle. The radar sensor operates at a frequency of 77 GHz and has a range of up to 200 meters, while the camera sensor is a high-resolution, monocular camera with a 30-degree field of view. 
 Operational Steps: 4. **Distance and Speed Calculation**: The system calculates the distance and speed of the target vehicle, using data from the radar and camera sensors, as well as GPS and IMU data. 
 Necessary Conditions: 2. **Road Conditions**: The ACC system requires a clear, dry road surface to operate effectively. 
 User Interface and Controls: 3. **Distance Setting**: The driver sets the desired distance from the target vehicle using a button on the steering wheel. 
 User Interface and Controls: 2. **Set Speed Button**: The d

In [ ]:
def generate_combined_paragraph(system, description, missing_items):

    system_prompt = """
    You are an assistant that helps in reformulating technical descriptions and identifying missing parts.
    Your task is to:
    1. Identify and list the missing parts based on the provided 'Missing_Items'. Don't change the name of sections with refomulation of rows content.
    2. Reformulate the given technical description with the missing items integrated to ensure clarity and coherence while maintaining the original meaning. Keep it as bullet points in same general structure.
    3. Present the output as a single paragraph that includes both the missing parts and the reformulated description. whitout any starters or explanation. give direct response.
    The input will be a System name, a list of missing items and  a technical description. The output should be a single paragraph in the following format:
    "It looks like you missed some important details in your initial response:\n
    [Missing parts here]

    Let me provide you with a complete description of the [system name] system:\n
    [Reformulated description here]"

    The description should include the following sections:

        # Technical Details
            - [Technical details]

        # Operational Steps
            - [Operational steps]

        # Interactions with Components and Systems
            - [Interactions]

        # Necessary Conditions
            - [Conditions]

        # User Interface and Controls
            - [Interface elements]

        # Fault Detection and Management
            - [Fault managements]

        # Performance Metrics
            - [Performance metrics]

        # Regulatory Compliance
            - [Regulatory aspects]

        # Integration with Advanced Features
            - [Integrations]

        Please ensure the description is detailed, technical, and comprehensive.
    """
    if not missing_items:
        missing_info_formatted = "No parts were missing."
    else:
        missing_info_formatted = missing_items

    user_prompt = f"""
    System name: {system}
    Description:
    {description}

    Missing_Items:
     - {missing_info_formatted}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7
    )

    result = response.choices[0].message.content
    return result

# Function to process each row
def process_row(row):
    print("Generating combined paragraph...")
    system = row['System']
    description = row['Description Modified']
    missing_items = row['Parsed Missing Rows']
    combined_paragraph = generate_combined_paragraph(system, description, missing_items)
    print("Paragraph generation successful.")
    return combined_paragraph


In [ ]:
import time

for index, row in df.iterrows():
    response = process_row(row)
    df.at[index, 'Combined_Paragraph'] = response
    time.sleep(0.2)

Generating combined paragraph...
Paragraph generation successful.


<ipython-input-43-a9c0feb9071a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'Combined_Paragraph'] = response


Generating combined paragraph...
Paragraph generation successful.
Generating combined paragraph...
Paragraph generation successful.
Generating combined paragraph...
Paragraph generation successful.
Generating combined paragraph...
Paragraph generation successful.
Generating combined paragraph...
Paragraph generation successful.
Generating combined paragraph...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jfz8gd6zeeq9vq125fys37jm` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499216, Requested 2308. Please try again in 4m23.267999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
df.head()

In [ ]:
df.shape

np.int64(94)

In [ ]:
df.to_csv('adas_systems_step-02.csv')

In [ ]:
from IPython.display import display, Markdown

display(Markdown(df['Combined_Paragraph'][900]))